<a href="https://colab.research.google.com/github/JairEsc/Mat_Apl/blob/main/MN_examen1_ejercicio2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from numba import njit
@njit  
def factorizar_LU_numba(matriz):
    n=len(matriz)
    L=np.zeros((n,n))
    U=np.zeros((n,n))
    for j in range(n):
        for i in range(j,n):
            sum=0
            for k in range(i):
                sum+=L[i,k]*U[k,j]
            L[i,j]=matriz[i,j]-sum
            
            if(i>j):
                sum=0
                for k in range(j):
                    sum+=L[j,k]*U[k,i]
                U[j,i]=(matriz[j,i]-sum)/L[j,j]
            else: 
                U[i,j]=1 
                if(L[i,i]==0):
                    print("Error en la construcción de L")
    return([L,U])
        

In [ ]:
@njit 
def superior_numba(list,b):
  n=len(list)
  X=np.zeros(n)
  X[n-1]=b[n-1]/list[n-1,n-1]
  for i in range(1,n):#len(list) es el número de renglones de la matriz
    acum=0
    for j in range(i):#0<=j<=i-1 
      acum+=X[n-1-j]*list[n-i-1,n-1-j]
    X[n-1-i]=((b[n-1-i]-acum)/list[n-1-i,n-1-i])
    
  return X
@njit
def inferior(list,b):#list debe contener las entradas de la matriz triangular.
#Será aceptable introducir un arreglo de arreglos de distintos tamaños, para solo introducir las entradas no cero.
  X=[b[0]/list[0][0]]
  for i in range(1,len(list)):#len(list) es el número de renglones de la matriz
    acum=0
    for j in range(i):#0<=j<=i-1 
      acum+=X[j]*list[i][j]
    X.append((b[i]-acum)/list[i][i])
  return X
def solve(L_m,U_m,b):
    y=inferior(L_m,b)#Resolver Ly=b
    return(superior_numba(U_m,y))#Resolver Ux=y

In [ ]:
A=np.loadtxt("A_1000.mtx",skiprows=1)
L,U=factorizar_LU_numba(A)
#Verificacion
def matrices_iguales(A,B,tol):#Funcion que verifica que dos matrices sean iguales salvo una tolerancia
    n=len(A)
    boolean=True
    if(n!=len(B)):
        print("ni siquiera son del mismo tam.")
    for i in range(n):
        if(boolean==True):
            for j in range(n):
                if(abs(B[i,j]-A[i,j])>tol):
                    boolean=False;
                    break;
        else:
            break;
    return(boolean)
print("La descomposicion es correcta con tolerancia 10**(-6): ",matrices_iguales(A,np.dot(L,U),tol=10**(-6)))
A_inv=[]
for i in range(1000):
    b=np.zeros(1000)
    b[i]=1
    A_inv.append(solve(L_m=L,U_m=U,b=b))
identidad_approximada=np.dot(A,np.array(A_inv).T)
identidad=np.identity(1000)
print("La inversa es correcta con tolerancia 10**(-6): ",matrices_iguales(identidad,identidad_approximada,tol=10**(-6)))

La descomposicion es correcta con tolerancia 10**(-6):  True
La inversa es correcta con tolerancia 10**(-6):  True


In [144]:
#Guardar la matriz
fo = open(f"A_inverse.txt", "w")
fo.write(str(1000)+" "+str(1000)+"\n")#Add size
np.savetxt(fo, np.array(A_inv).T, fmt='%1.9e', comments='')

fo.close()

In [127]:
b=np.loadtxt("b_1000.vec",skiprows=1)
x_sol=np.dot(np.array(A_inv).T,b)
print("error ||Ax-b|| : ",np.linalg.norm(np.dot(A,x_sol)-b))

error ||Ax-b|| :  1.042980169448857e-12


In [147]:
#Guardar la solucion
fo = open(f"x_sol.txt", "w")
fo.write(str(1000)+"\n")#Add size
np.savetxt(fo, x_sol, fmt='%1.9e', comments='')
fo.close()